In [3]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
conn = psycopg2.connect(
    host="localhost",
    database="gpx",
    user="root",
    password="root"
)

In [8]:
query = """
WITH lines AS (
    SELECT
        b.name,
        ST_MakeLine(
            ST_SetSRID(ST_MakePoint(a.longitude, a.latitude), 4326),
            ST_SetSRID(ST_MakePoint(b.longitude, b.latitude), 4326)
        ) AS geom
    FROM start a
    CROSS JOIN destinations b
),
points AS (
    SELECT
        id,
        name,
        ST_SetSRID(ST_MakePoint(longitude, latitude), 4326) AS geom
    FROM points_of_interest
)

SELECT
    a.name AS destination,
    b.name AS point_name,
    ST_Distance(
        ST_Transform(b.geom, ST_SRID(a.geom))::geography,
        a.geom::geography
    ) AS distance_to_line_in_meters
FROM lines a
CROSS JOIN points b;
"""

with conn.cursor() as cur:
    cur.execute(query)
    rows = cur.fetchall()

df = pd.DataFrame(rows, columns=["destination_name", "point_name", "distance"])

df.head(3)

,destination_name,point_name,distance
0,Merlion,Tanjong Rhu,284.681816
1,Merlion,Tanjong Rhu View,329.232245
2,Merlion,Tanjong Rhu Bridge,195.690482
